# Line chart (benchmark analysis)

In [15]:

import pandas as pd
import vizro.models as vm
import vizro.plotly.express as px
from vizro import Vizro
from vizro.tables import dash_ag_grid
from vizro.actions import export_data, filter_interaction
import plotly.graph_objects as go
from vizro.models.types import capture

Vizro._reset()
# Load the data

gapminder = pd.read_csv('uni.csv',
                 header=0,
                 usecols=["University Name", "Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"])

# y values for the direct labels to show the national rankings
y_arts=[1, 3, 4, 2, 3, 3, 5, 3]

hovertext=[
            '1st place nationally, Ranked 113 in the world by THE 2024', 
            '=3rd place nationally, Ranked 350.5 in the world by THE 2024', 
            '4th place nationally, Ranked 450.5 in the world by THE 2024',
            '2nd place nationally, Ranked 188 in the world by THE 2024',
            '=3rd place nationally, Ranked 350.5 in the world by THE 2024',
            '=3rd place nationally, Ranked 350.5 in the world by THE 2024',
            '5th place nationally, Ranked 550.5 in the world by THE 2024',
            '=3rd place nationally, Ranked 350.5 in the world by THE 2024'
        ],


@capture("graph")
def barchart(data_frame, x, y, title=None, text=None): 
    fig_bar = px.bar(data_frame=data_frame, x=x, y=y, title=title, text=text)
    
    # fig_bar.update_layout(
    #     title=go.layout.Title(
    #         text="NZ Uni rankings",
    #         font=dict(
    #             family="Tahoma, sans-serif, bold",
    #             size=25,

    #         ),
    #         xref="paper",
    #         x=0
    #         ),
    
    #         legend=dict(
    #             font=dict(
    #                 family="Tahoma, sans-serif",
    #                 size=12,
    #                 # color="black"
    #     )
    # ),
  
    # annotations=[
    #     dict(
    #         xref='paper', 
    #         yref='paper',
    #         x=0, 
    #         y=-0.2,
    #         showarrow=False,
    #         text="Source: <a href='https://www.topuniversities.com/universities/university-auckland'>QS Rankings</a>",
    #         font=dict(
    #             family="Tahoma, sans-serif",
    #             size=12,
    #         )
    #     )
    # ],
        
    #     yaxis=dict(autorange='reversed'))
    
    return fig_bar
    
def create_subject():
    """Function returns a page to perform analysis on university level."""

   
    # Benchmark analysis
    page_subject = vm.Page(
        title="Uni rankings by subject",
        description="Discovering how different NZ universities are ranked through the years",
        layout=vm.Layout(grid=[[0]]),
        components=[
            vm.Graph(
                id="barchart",
                figure=barchart(
                    gapminder,
                    x="University Name",
                    y="Arts and humanities ranking THE2024",
                    text=y_arts,
                ),
                
        controls=[
            vm.Parameter(
                targets=["barchart.y"],
                selector=vm.Dropdown(
                    options=["Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"], multi=False, value="Arts and humanities ranking THE2024", title="Choose subject field"
                ),
            ),
            
        ],
    )])
        
    return page_subject

dashboard = vm.Dashboard(
    pages=[
        create_subject()    
    ],)

IS_JUPYTERLAB = 'true'

if not IS_JUPYTERLAB:
    app = Vizro().build(dashboard)
    server = app.dash.server
    
    if __name__ == "__main__":  
        app.run(port=8080)
else:
    Vizro(assets_folder="assets").build(dashboard).run(port=8083)



ValidationError: 1 validation error for Parameter
targets -> 0
  Target barchart not found in model_manager. (type=value_error)